In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")
def img_caption(img_path):
  raw_image = Image.open(img_path)
  inputs = processor(raw_image, return_tensors="pt").to("cuda")
  out = model.generate(**inputs)
  return processor.decode(out[0], skip_special_tokens=True)

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
import re

def extract_twitter_urls(text):
    pattern = r'https://pbs\.twimg\.com/media/[A-Za-z0-9_-]+\?format=jpg&name=small'
    matches = re.findall(pattern, text)
    return matches[0] if matches else None


In [ ]:
import requests

def download_and_save_image(url, destination_path):
      response = requests.get(url, stream=True)
      if response.status_code == 200:
          with open(destination_path, 'wb') as file:
              for chunk in response.iter_content(chunk_size=128):
                  file.write(chunk)
      else :
        return

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/demo_tests.xlsx')
df.to_csv('demo.csv', index=False)
test_data=pd.read_csv('demo.csv')
df = pd.DataFrame(test_data)

In [ ]:
import os
os.mkdir("Folder_path")

In [ ]:

folder_path="Folder_path/"
for i in range(len(df['media'])):
  url=extract_twitter_urls(df['media'][i])
  if url==None:
    continue
  download_and_save_image(url,folder_path+str(i)+'.jpg')

In [ ]:
def extract_integers_from_string(file_name):
    integers = re.findall(r'\d+', file_name)
    integers = [int(num) for num in integers]
    return integers[0]

In [ ]:
df['captions']=pd.Series(dtype='str')

In [ ]:
directory2 = 'Folder_path'
for filename in os.listdir(directory2):
  f=os.path.join(directory2, filename)
  idx=extract_integers_from_string(filename)
  df['captions'][idx]=img_caption(f)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
<ipython-input-47-8a28cd05e5f3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['captions'][idx]=img_caption(f)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
for i in range(len(df)):
  if type(df['captions'][i])==float:
    df['captions'][i]=''

In [ ]:
df['Prediction']=pd.Series(dtype='str')

In [ ]:
# the relative path of the zip folder should be checkpoint/run1/
!unzip '/content/drive/MyDrive/checkpoint.zip'

NotImplementedError: ignored

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.utils.data import Dataset, DataLoader
from transformers import DataCollatorForLanguageModeling
import torch

In [ ]:
!pip install gpt_2_simple

  Preparing metadata (setup.py) ... done
  Created wheel for gpt_2_simple: filename=gpt_2_simple-0.8.1-py3-none-any.whl size=24557 sha256=4f23585799d59e88403b3aac6c20b7181925eb598d35e222cc15ba77970e481f
  Stored in directory: /root/.cache/pip/wheels/df/6a/fe/10d3223f78d1ac3e4c83bb4c5e2d28dfb1789c2fb4cc7ea8d0
Successfully built gpt_2_simple


In [ ]:
import tensorflow as tf

In [ ]:
import gpt_2_simple as gpt2

sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)

Loading checkpoint checkpoint/run1/model-500


In [ ]:
img_caption(image_path)

In [ ]:
gpt2_generated_tweet = gpt2.generate(sess,
              prefix="date : 2020-10-22 21:37:01+00:00 Likes : 6 inferred company : Target  Username : Targetstore Captions : "+"a man holding a bouque of flowers",
              length=50,
              temperature=0.7,
              truncate = '<|endoftext|>',
              return_as_list=True
              )

In [ ]:
gpt2_generated_tweet[0].split('\n')[0]

'date : 03-09-2019  Likes : 178 inferred company : Target  Username : Targetstore Captions : a man holding a bouque of flowers tweets: time Valentines Day ! Celebrate life , love , read life ! # HappyZDay'

In [ ]:
decoded_str=gpt2_generated_tweet[0].split('\n')[0]
pattern = r'tweets(.*)'
match = re.search(pattern, decoded_str)
extracted_text = match.group(1).strip()

In [ ]:
extracted_text

': time Valentines Day ! Celebrate life , love , read life ! # HappyZDay'

In [ ]:
for i in range(len(df)):
  prompt= 'date: '+df['date'][i]+' likes: '+str(df['likes'][i])+' inferred comapny: '+df['inferred company'][i]+' username: '+df['username'][i]+' caption: '+img_caption("/content/Folder_path/"+str(i)+".jpg")
  decoded_str=gpt2.generate(sess,
              prefix=prompt,
              length=50,
              temperature=0.8,
              truncate = '<|endoftext|>',
              return_as_list=True
              )

  print(decoded_str)
  # pattern = r'tweets(.*)'
  # match = re.search(pattern, decoded_str)
  # extracted_text = match.group(1).strip()

['date: 2020-10-22 21:37:01+00:00 likes: 633 inferred comapny: alaska airlines username: AlaskaAir caption: a man and woman in the airplane with masks on tweets: known # AfriForAll Saga Sulu American Airlines stock market cap unraveling first woman hold senior executive position\ndate: 2018-04-20 21:01:02 Likes: 310 captions: a woman with a short haircut tweets']
['date: 2019-10-24 10:06:42+00:00 likes: 1960 inferred comapny: nokia username: NokiaMobile caption: nokia z2 with nokia 621 text: nokia hits desktop world # 1addamsph\ndate: 2020-01-10 18:21:01 Likes: 340 captions: a black and white photo of a man sitting on a couch tweets: senior manager , clicks']
['date: 2019-01-01 15:19:10+00:00 likes: 728 inferred comapny: ralph lauren username: RalphLauren caption: a man in a suit and bow tie standing in a hallway tweets: colleagues , love passing time protecting customersefe . hard times keeping pace # GoodTogether\ndate: 2018-08-18 14:00:57 Likes: 228 captions: the podcast podcast pod